# Real-World Applications: TF-IDF
In this task Hadoop Streaming is used to process Wikipedia articles dump (/data/wiki/en_articles_part).

The purpose of this task is to calculate tf*idf for each pair (word, article) from the Wikipedia dump. Apply the stop words filter to speed up calculations. Term frequency (tf) is a function depending on a term (word) and a document (article):

tf(term, doc_id) = Nt/N,

where Nt - quantity of particular term in the document, N - the total number of terms in the document (without stop words)

Inverse document frequency (idf) is a function depends on a term:

idf(term) = 1/log(1 + Dt),

where Dt - number of documents in the dataset with the particular term.

You can find more information here: https://en.wikipedia.xn--org/wiki/Tfidf-q82h but use just the formulas mentioned above.

Dataset location: /data/wiki/en_articles_part
Stop words list is in ‘/datasets/stop_words_en.txt’ file.
Format: article_id

To parse the articles don’t forget about Unicode (even though this is an English Wikipedia dump, there are many characters from other languages), remove punctuation marks and transform words to lowercase to get the correct quantities. To cope with Unicode we recommend to use the following tokenizer:

Output: tf*idf for term=’labor’ and article_id=12

The result on the sample dataset:

0.000351

**Hint**: all Wikipedia article_ids are greater than 0. So you can use a dummy article_id=0 to calculate the number of documents with each term.

In [25]:
%%writefile mapper_tfidf.py

import sys
import re
import collections

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open("stop_words_en.txt", "r") as f:
    stop_words = f.read().splitlines()
    
def cleanup(words):
    return [word.lower().strip() for word in words if (word.lower() not in stop_words)]

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        words = cleanup(words)
        
        words_counter = collections.Counter(words)
        words_total = sum(words_counter.values())

        for word, count in sorted(words_counter.items()):
            tf = float(count)/float(words_total)
            print("{}\t{}\t{:f}".format(word, article_id, tf))
             
    except Exception as e:
        continue

Overwriting mapper_tfidf.py


In [26]:
%%writefile reducer_tfidf.py

import sys
import math

current_word = None
articles = {}

for line in sys.stdin:
    try:
        word, article_id, tf = line.strip().split('\t', 2)
        tf = float(tf)
        
        if current_word != word:
            if current_word:
                idf = float(1)/math.log(1 + len(articles))
                for article, tff in articles.items():
                    print("{}\t{}\t{:f}".format(current_word, article, tff*idf))
            
            current_word = word
            articles = {}
        
        articles[article_id] = float(tf)
    
    except Exception as e:
        continue
        
if current_word:
    idf = float(1)/math.log(1 + len(articles))
    for word, tf in articles.items():
        print("{}\t{}\t{:f}".format(current_word, article, tff*idf))

Overwriting reducer_tfidf.py


In [28]:
%%bash

OUT_DIR="8_tfidf"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="8_TF-IDF" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -files mapper_tfidf.py,reducer_tfidf.py,/datasets/stop_words_en.txt \
    -mapper "python mapper_tfidf.py" \
    -reducer "python reducer_tfidf.py" \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 

hdfs dfs -cat ${OUT_DIR}/part* | grep -w "labor" | grep -w "12" | cut -f 3

0.000351


20/11/14 19:39:35 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/11/14 19:39:35 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/11/14 19:39:35 INFO mapred.FileInputFormat: Total input files to process : 1
20/11/14 19:39:35 INFO mapreduce.JobSubmitter: number of splits:2
20/11/14 19:39:35 INFO Configuration.deprecation: yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
20/11/14 19:39:35 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1605371019884_0015
20/11/14 19:39:35 INFO conf.Configuration: resource-types.xml not found
20/11/14 19:39:35 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
20/11/14 19:39:35 INFO resource.ResourceUtils: Adding resource type - name = memory-mb, units = Mi, type = COUNTABLE
20/11/14 19:39:35 INFO resource.ResourceUtils: Adding resource type - name = vcores, units = , type = COUNTABLE
20/11/14 19:39:35 INFO impl.Ya